In [196]:
import numpy as np
import pandas as pd
import tqdm
from ghcn import load_daily
from glob import glob
import scipy.stats as stats

from statsmodels.stats.descriptivestats import sign_test
# Data files
files = sorted(glob('ghcnd_small/*.dly'))

In [2]:
# Load data about stations (we only the station ID and latitude)
# that are in the northern hemisphere (a.k.a. latitude > 0)
north_stations = pd.read_fwf("ghcnd-stations.txt", header=None, usecols=[0, 1])
north_stations.columns = ["station_id", "latitude"]
north_stations = north_stations[north_stations["latitude"] > 0]
north_stations.set_index("station_id", inplace=True)


In [48]:
data_all_t = []
for filename in tqdm.tqdm(files):
    # Get the station name from the filename
    station_name = filename.split(sep='/')[-1][:-4]
    # Get the latitude of the station if it's in the dictionary (if not, it's -1)
    try:
        latitude = north_stations['latitude'][station_name]
    except:
        latitude = -1
    # Only load this file if the station is in the northern hemisphere (a.k.a. having latitude > 0)
    if latitude < 0:
        continue
    
    # All the data for one station
    df_temp = pd.DataFrame.from_records(load_daily(filename))

    # Extract the temperature data
    filter_ = np.logical_or(df_temp["element"] == "TMIN", df_temp["element"] == "TMAX")
    
    temperatures = df_temp[filter_]

    # Delete unnecessary columns
    temperatures = temperatures.drop(columns=["measurement", "quality", "source"])

    data_all_t.append(temperatures)

    del filename, station_name, latitude, df_temp, filter_, temperatures

100%|██████████| 1000/1000 [01:19<00:00, 12.53it/s]


In [49]:
data_all_t = pd.concat(data_all_t)

# Delete missing data
df_t = data_all_t[data_all_t["value"] != -9999]
df_t

,station_id,year,month,element,day,value
0,AGM00060490,1957,1,TMAX,1,178
1,AGM00060490,1957,1,TMAX,2,150
2,AGM00060490,1957,1,TMAX,3,161
3,AGM00060490,1957,1,TMAX,4,172
4,AGM00060490,1957,1,TMAX,5,172
...,...,...,...,...,...,...
50821,VMM00048808,2021,3,TMIN,13,195
50822,VMM00048808,2021,3,TMIN,14,196
50823,VMM00048808,2021,3,TMIN,15,211
50824,VMM00048808,2021,3,TMIN,16,212


In [50]:
# ===========================================================
# Compute each station's annual mean temperature
# ===========================================================

# For each station and for each day, compute the midpoint temperature by
# averaging the min and max temperatures
mid_temps = df_t.where(np.logical_or(df_t["element"] == "TMIN",
                            df_t["element"] == "TMAX")).groupby(by=["station_id", "year", "month", "day"]).mean().reset_index()

# For each station and for each year, compute the average temperature across that year
temps = mid_temps.groupby(by=["station_id", "year"]).mean()["value"].reset_index()

# All temperatures are in tenths of degree Celsius, so divide by 10 to get
# actual Celsius temperatures
temps["value"] /= 10

# Convert year to int type
temps["year"] = temps["year"].astype(int)

temps = temps.set_index(["station_id", "year"])
temps.rename(columns={"value": "temp"}, inplace=True)
print(temps.shape)

(11486, 1)


In [51]:
temps

temp
station_id  year           
AGM00060490 1957  17.622492
            1958  18.110000
            1959  18.911058
            1960  19.716121
            1961  20.243947
...                     ...
VMM00048808 2017  20.512021
            2018  21.082609
            2019  23.860252
            2020  20.951423
            2021  16.163077

[11486 rows x 1 columns]

In [7]:
# We only care about stations in the northern hemisphere (latitude > 0)
# So go through the list of files, and only load files corresponding to
# stations in the northern hemisphere.
data_all = []
for filename in tqdm.tqdm(files):
    # Get the station name from the filename
    station_name = filename.split(sep='/')[-1][:-4]
    # Get the latitude of the station if it's in the dictionary (if not, it's -1)
    try:
        latitude = north_stations['latitude'][station_name]
    except:
        latitude = -1
    # Only load this file if the station is in the northern hemisphere (a.k.a. having latitude > 0)
    if latitude < 0:
        continue
    
    # All the data for one station
    df = pd.DataFrame.from_records(load_daily(filename))

    # Extract the temperature data
    filter_ = np.logical_and(np.logical_and(np.logical_and(np.logical_and(
                                        df["element"] != "TMIN",
                                        df["element"] != "TMAX"), 
                                        df["element"] != "PRCP"), 
                                        df["element"] != "SNOW"),
                                        df["element"] != "TAVG")
    
    temperatures = df[filter_]

    # Delete unnecessary columns
    temperatures = temperatures.drop(columns=["measurement", "quality", "source"])

    data_all.append(temperatures)

    del filename, station_name, latitude, df, filter_, temperatures
print("Reading files... DONE")

# Combine all the dataframes
data_all = pd.concat(data_all)


100%|██████████| 1000/1000 [01:25<00:00, 11.65it/s]


Reading files... DONE


In [15]:
nn_data = data_all[data_all.value != -9999].dropna()
df = nn_data[nn_data.element != 'SNWD']

In [16]:
df

,station_id,year,month,element,day,value
774,ASN00001031,1998,10,DAPR,31,16
805,ASN00001031,1998,10,DWPR,31,5
836,ASN00001031,1998,10,MDPR,31,770
897,ASN00001031,1998,11,DAPR,30,4
928,ASN00001031,1998,11,DWPR,30,4
...,...,...,...,...,...,...
95554,USW00094724,2021,3,WSF5,13,188
95555,USW00094724,2021,3,WSF5,14,183
95556,USW00094724,2021,3,WSF5,15,188
95557,USW00094724,2021,3,WSF5,16,85


In [20]:
def compute_yearly_metrics(metrics):
    metrics_data = []
    for metric in tqdm.tqdm(metrics):
        # Extract the data
        data = df.where(df['element'] == metric).dropna()
        
        # Delete missing data, and unnecessary columns
        data = data[data["value"] != -9999]

        # For each station and for each year, compute the average metric across that year
        result = data.groupby(by=["station_id", "year"]).mean()["value"].reset_index()

        # Convert year to int type
        result["year"] = result["year"].astype(int)
        
        result = result.set_index(["station_id", "year"])
        result.rename(columns={"value": metric}, inplace=True)
        
        metrics_data.append(result)

    return metrics_data

# Join the temperature and metric data (on the index a.k.a. the year column)
metrics_list = df['element'].unique()

# data = pd.concat(data, axis=1, join="inner", ignore_index=False)

100%|██████████| 72/72 [01:14<00:00,  1.04s/it]

[(3434, 1), (1282, 1), (3825, 1), (12, 1), (12, 1), (12, 1), (12, 1), (250, 1), (106, 1), (117, 1), (1052, 1), (776, 1), (1904, 1), (491, 1), (494, 1), (403, 1), (1141, 1), (378, 1), (2086, 1), (911, 1), (1342, 1), (190, 1), (4143, 1), (1007, 1), (184, 1), (256, 1), (241, 1), (107, 1), (107, 1), (52, 1), (52, 1), (350, 1), (109, 1), (11, 1), (100, 1), (101, 1), (43, 1), (43, 1), (61, 1), (61, 1), (9, 1), (9, 1), (10, 1), (10, 1), (145, 1), (13, 1), (185, 1), (71, 1), (132, 1), (131, 1), (132, 1), (131, 1), (41, 1), (2, 1), (44, 1), (35, 1), (1, 1), (1, 1), (36, 1), (37, 1), (7, 1), (6, 1), (6, 1), (33, 1), (47, 1), (17, 1), (6, 1), (36, 1), (4, 1), (6, 1), (28, 1), (28, 1)]


In [119]:
data = [temps] + compute_yearly_metrics(metrics_list)
print([x.shape for x in data])


100%|██████████| 72/72 [01:18<00:00,  1.09s/it]

[(11486, 1), (3434, 1), (1282, 1), (3825, 1), (12, 1), (12, 1), (12, 1), (12, 1), (250, 1), (106, 1), (117, 1), (1052, 1), (776, 1), (1904, 1), (491, 1), (494, 1), (403, 1), (1141, 1), (378, 1), (2086, 1), (911, 1), (1342, 1), (190, 1), (4143, 1), (1007, 1), (184, 1), (256, 1), (241, 1), (107, 1), (107, 1), (52, 1), (52, 1), (350, 1), (109, 1), (11, 1), (100, 1), (101, 1), (43, 1), (43, 1), (61, 1), (61, 1), (9, 1), (9, 1), (10, 1), (10, 1), (145, 1), (13, 1), (185, 1), (71, 1), (132, 1), (131, 1), (132, 1), (131, 1), (41, 1), (2, 1), (44, 1), (35, 1), (1, 1), (1, 1), (36, 1), (37, 1), (7, 1), (6, 1), (6, 1), (33, 1), (47, 1), (17, 1), (6, 1), (36, 1), (4, 1), (6, 1), (28, 1), (28, 1)]


In [155]:
# s_data = [x if x.shape[0] > 400 else _ for x in data]
data_df = pd.concat(data, axis=1, ignore_index=False)

In [166]:
factor_df_list = []
for col in data_df.columns:
    if col == 'temp':
        continue
    factor_df_list.append(data_df[['temp', col]])
        
factor_df_list_adj = []
for df_ in factor_df_list:
    tr_df = df_.dropna().reset_index(level="year")
    year_column = tr_df["year"]
    all_years = year_column.unique()
    all_years.sort()
    if len(all_years) <= 2:
        continue
    all_adj_years = np.lib.stride_tricks.sliding_window_view(all_years, 2)
    factor_df_list_adj.append((tr_df, all_adj_years)) if df_.dropna().shape[0] > 100 else _



In [ ]:
factor_df_list_adj[0]

In [203]:
alpha = 0.05

outputs = []
for df_adj in factor_df_list_adj:
    df_, all_adj_years = df_adj
    num_sig_factor = 0
    
    for (year1, year2) in tqdm.tqdm(all_adj_years):
        # Get station data for both years
        data_year1 = df_[df_['year'] == year1]
        data_year2 = df_[df_['year'] == year2]
        
        # Get specific columns
        temp_year1 = data_year1["temp"]
        temp_year2 = data_year2["temp"]
        factor_year1 = data_year1[data_year1.columns[-1]]
        factor_year2 = data_year2[data_year2.columns[-1]]

        # Inner join on station id to find observations common between both years, and
        # then compute the differences
        temp_year_diff = pd.DataFrame(temp_year1).join(temp_year2, how="inner",
                                                         lsuffix="_{}".format(year1),
                                                         rsuffix="_{}".format(year2)).diff(axis=1).iloc[:, 1]
        factor_year_diff = pd.DataFrame(factor_year1).join(factor_year2, how="inner",
                                                         lsuffix="_{}".format(year1),
                                                         rsuffix="_{}".format(year2)).diff(axis=1).iloc[:, 1]    
        # Only looking at cases where there are at least 5 values. 
        # The sign test is able to handle a small number of values 
        # but we want to cap at 5 values minimum.
        if factor_year_diff.shape[0] < 5 or temp_year_diff.shape[0] < 5:
            continue
        # Count the number of differences that are positive
        #
        # These three values will serve as our test statistics for a sign test on temperature,
        # a sign test on precipitation, and a sign test on snowfall, respectively
        test_stat_temp = np.count_nonzero(temp_year_diff > 0)
        test_stat_factor = np.count_nonzero(factor_year_diff > 0)

        # Convert each test statistic into a p-value using the binomial test. In this case, each
        # test statistic is defined as the number of successes (a.k.a. the number of stations
        # for which temperature increased, precipitation increased, and snowfall increased,
        # respectively) out of all stations
        p_value_temp = stats.binomtest(k=test_stat_temp, n=temp_year_diff.size, alternative='two-sided').pvalue
        p_value_factor = stats.binomtest(k=test_stat_factor, n=factor_year_diff.size, alternative='two-sided').pvalue

        # If the p-values for temperature and precipitation are BOTH significant, then it means
        # that between year1 and year2, temperature and precipitation BOTH had a statistically
        # significant increase
        if p_value_temp < alpha and p_value_factor < alpha:
            num_sig_factor += 1
    outputs.append((df_.columns[-1], num_sig_factor))


100%|██████████| 24/24 [00:00<00:00, 141.91it/s]


In [140]:
len(factor_df_list)

0

In [61]:
dapr.dropna()

temp      DWPR
station_id  year                     
ASN00015089 1976  25.184211  3.000000
            1978  25.660606  2.000000
            1980  30.145588  2.666667
ASN00018120 1983  16.493726  2.000000
            1988  18.124344  2.333333
            2004  17.732787  3.000000
            2005  18.046027  2.000000
            2006  17.490000  2.000000
ASN00030024 1990  24.426374  2.000000
            1999  24.401389  2.000000
ASN00040852 1993  18.451567  2.750000
            1994  19.763793  2.250000
            1996  18.460792  2.000000
            1997  18.382877  2.000000
            1999  29.583871  2.000000
ASN00086097 1965  14.190500  2.571429
            1966  14.193750  2.409091
            1967  14.299705  2.500000
            1973  14.230249  2.000000
            1976  14.169452  2.000000
ASN00091007 1971  13.268508  1.666667
            1972  13.195219  1.000000
            1975  12.956849  2.000000
            1976  12.962842  2.000000
            1978  12.652358  2.000000
ASN00091250 1986  14.002381  1.000000
            1987  10.918293  1.000000
            1988  12.287273  1.000000
ASN00096003 1976   7.862295  2.000000
            1985   7.668219  2.000000